### Task1b Regression with Feature Transformation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('./train.csv')
df.head(5)

,Id,y,x1,x2,x3,x4,x5
0,0,-7.912551,1.276266,-0.854628,1.623901,2.145311,2.037190
1,1,-5.728060,2.886639,0.888302,0.637899,1.148675,0.562217
2,2,-29.524046,3.171257,2.152310,-0.818812,0.861951,1.539840
3,3,-22.108731,-0.775282,2.314877,0.526552,-0.092230,0.749973
4,4,-15.234738,0.017706,2.031269,1.491334,0.553353,0.193640


In [3]:
y = df['y']
X = df._drop_axis(['Id','y'], axis=1)

#switch to numpy for the functions
y = np.array(y)
X = np.array(X)

#useful variables
num_rows, num_cols = X.shape 

In [4]:
#feature transormations
X_quad = np.square(X)
X_exp = np.exp(X)
X_cos = np.cos(X)
X_const = np.ones((num_rows, 20))

#concatenate all these transformations into transformed X
print(X.shape, X_quad.shape, X_exp.shape, X_cos.shape, X_const.shape)
X = np.concatenate((X, X_quad, X_exp, X_cos, X_const),axis=1)

#For some reason, it is impossible to append vector to numpy array, only matrix of same col
X_transformed = X[:,0:21]
print(X_transformed.shape)

(900, 5) (900, 5) (900, 5) (900, 5) (900, 20)
(900, 21)


In [5]:
#Set up regression
k = 10
weights = np.empty(21) #9 folds so 9 models, 21 weights (w1...w21)
i = 0
j = 0
rmse = 0
min_rmse = 15343 #arbitrarily large
min_lambda = 0
lambdas = [.03, .05, .09, .1, .3, .5, .7, .9, 1, 3, 9]

#Test if column of ones added
X_transformed[0,20]

1.0

### Now k-fold cross validation

In [6]:
#Split into k folds, each fold contains 900/k rows
kf = KFold(k)

#for l in lambdas: 
for train_index, test_index in kf.split(X_transformed):
    X_train, X_test = X_transformed[train_index], X_transformed[test_index]
    y_train, y_test = y[train_index], y[test_index]

    lasso = Lasso(alpha=.5)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)

    rmse = mean_squared_error(y_pred, y_test)**0.5

    #if this rmse is lower, store the weights of this fitted model
    if(rmse < min_rmse):
        min_rmse = rmse
        weights = lasso.coef_
        print('RMSE: ', min_rmse)

        i = i + 1
    
print('Lambda: ', min_lambda)
print('Lowest RMSE: ', min_rmse)

RMSE:  10.83787022080987
RMSE:  9.906227613815997
Lambda:  0
Lowest RMSE:  9.906227613815997


#### Already found min rmse for each k-fold step, now find min rmse for each lambda

In [7]:
np.savetxt("submit.csv", weights, delimiter=",")